In [1]:
# To be run after Taiwan-Data-Prep-Marriage
# Import Data handling/display libraries
import sys
import pandas as pd
import numpy  as np 
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.ensemble      import RandomForestClassifier
from sklearn.linear_model  import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics       import accuracy_score
from sklearn.metrics       import auc, roc_auc_score, roc_curve, classification_report, confusion_matrix
from IPython.display       import Markdown, display
# Import IBM's AI Fairness tooolbox
from aif360.datasets       import BinaryLabelDataset
from aif360.metrics        import BinaryLabelDatasetMetric
from aif360.metrics        import ClassificationMetric
from aif360.metrics.utils  import compute_boolean_conditioning_vector
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas, load_preproc_data_german
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
%matplotlib inline
# Warnings will be used to silence various model warnings for tidier output
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [2]:
Taiwan_df = pd.read_csv('./input/Taiwan-Credit-Card-Cleaned-Marriage.csv')

In [3]:
# Set privileged / unprivileged / favourable / unfavourable values
protected_attr = 'MARRIAGE'
priv_grp       = 1
unpriv_grp     = 2
lab            = 'DEFAULT'
fav_label      = 0
unfav_label    = 1

In [4]:
# Create the traoning and test splits
X = Taiwan_df.drop(lab,axis=1)
y = Taiwan_df[lab]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 101)

In [5]:
# Create a Binary Label Dataset to use with AIF360 APIs
# Favourable Label : DEFAULT = 1 (no default)
# Favourable Label : DEFAULT = 0 (Default)
Taiwan_bld = BinaryLabelDataset(df=pd.concat((X, y), axis=1),
                                  label_names=[lab], protected_attribute_names=[protected_attr],
                                  favorable_label=fav_label, unfavorable_label=unfav_label)
privileged_groups   = [{protected_attr: priv_grp}]   # Married folk (male or female)
unprivileged_groups = [{protected_attr: unpriv_grp}] # Single folk  (male or female)

In [6]:
Taiwan_bld_train, Taiwan_bld_test = Taiwan_bld.split([0.8], shuffle=True)

In [7]:
# Metric for the original dataset
Taiwan_metric_train = BinaryLabelDatasetMetric(Taiwan_bld_train, 
                                               unprivileged_groups=unprivileged_groups,
                                               privileged_groups=privileged_groups)
#display(Markdown("#### Original Taiwan_bld training dataset"))
print("Train set: Difference in mean outcomes between unprivileged/privileged groups = %f" % Taiwan_metric_train.mean_difference())
Taiwan_metric_test = BinaryLabelDatasetMetric(Taiwan_bld_test, 
                                              unprivileged_groups=unprivileged_groups,
                                              privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged/privileged groups = %f" % Taiwan_metric_test.mean_difference())

Train set: Difference in mean outcomes between unprivileged/privileged groups = 0.027419
Test set: Difference in mean outcomes between unprivileged/privileged groups = 0.017548


In [8]:
#Scale the binary labelled datasets
min_max_scaler = MinMaxScaler()
Taiwan_bld_train.features = min_max_scaler.fit_transform(Taiwan_bld_train.features)
Taiwan_bld_test.features  = min_max_scaler.fit_transform(Taiwan_bld_test.features)

In [9]:
# Verify that the scaling does not affect the group label statistics
Taiwan_bld_metric_scaled_train = BinaryLabelDatasetMetric(Taiwan_bld_train, 
                          unprivileged_groups=unprivileged_groups,
                          privileged_groups=privileged_groups)
print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % Taiwan_bld_metric_scaled_train.mean_difference())
Taiwan_bld_metric_scaled_test = BinaryLabelDatasetMetric(Taiwan_bld_test, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)
print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % Taiwan_bld_metric_scaled_test.mean_difference())

Train set: Difference in mean outcomes between unprivileged and privileged groups = 0.027419
Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.017548


In [10]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to FALSE!
sess = tf.Session()
biased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess)

In [11]:
biased_model.fit(Taiwan_bld_train)

W0813 14:36:29.875608 10032 deprecation_wrapper.py:119] From c:\users\befaria\appdata\local\continuum\miniconda3\lib\site-packages\aif360\algorithms\inprocessing\adversarial_debiasing.py:138: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0813 14:36:29.877609 10032 deprecation_wrapper.py:119] From c:\users\befaria\appdata\local\continuum\miniconda3\lib\site-packages\aif360\algorithms\inprocessing\adversarial_debiasing.py:142: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0813 14:36:29.883607 10032 deprecation_wrapper.py:119] From c:\users\befaria\appdata\local\continuum\miniconda3\lib\site-packages\aif360\algorithms\inprocessing\adversarial_debiasing.py:87: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0813 14:36:31.277653 10032 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https:

epoch 0; iter: 0; batch classifier loss: 0.718564
epoch 1; iter: 0; batch classifier loss: 0.515099
epoch 2; iter: 0; batch classifier loss: 0.391685
epoch 3; iter: 0; batch classifier loss: 0.435972
epoch 4; iter: 0; batch classifier loss: 0.422793
epoch 5; iter: 0; batch classifier loss: 0.494179
epoch 6; iter: 0; batch classifier loss: 0.499144
epoch 7; iter: 0; batch classifier loss: 0.410303
epoch 8; iter: 0; batch classifier loss: 0.553528
epoch 9; iter: 0; batch classifier loss: 0.384676
epoch 10; iter: 0; batch classifier loss: 0.363092
epoch 11; iter: 0; batch classifier loss: 0.402084
epoch 12; iter: 0; batch classifier loss: 0.459060
epoch 13; iter: 0; batch classifier loss: 0.376888
epoch 14; iter: 0; batch classifier loss: 0.412601
epoch 15; iter: 0; batch classifier loss: 0.479588
epoch 16; iter: 0; batch classifier loss: 0.453669
epoch 17; iter: 0; batch classifier loss: 0.344656
epoch 18; iter: 0; batch classifier loss: 0.534104
epoch 19; iter: 0; batch classifier loss:

In [12]:
# Apply the plain model to test data
Taiwan_biased_train = biased_model.predict(Taiwan_bld_train)
Taiwan_biased_test  = biased_model.predict(Taiwan_bld_test)

In [13]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
Taiwan_metric_biased_train = BinaryLabelDatasetMetric(Taiwan_biased_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % Taiwan_metric_biased_train.mean_difference())

Taiwan_metric_biased_test = BinaryLabelDatasetMetric(Taiwan_biased_test, 
                            unprivileged_groups=unprivileged_groups,
                            privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % Taiwan_metric_biased_test.mean_difference())

display(Markdown("#### Plain model - without debiasing - classification metrics"))
Taiwan_classified_metric_biased_test = ClassificationMetric(Taiwan_bld_test,Taiwan_biased_test,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % Taiwan_classified_metric_biased_test.accuracy())
TPR = Taiwan_classified_metric_biased_test.true_positive_rate()
TNR = Taiwan_classified_metric_biased_test.true_negative_rate()
bal_acc_biased_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_biased_test)
print("Test set: Disparate impact = %f" % Taiwan_classified_metric_biased_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % Taiwan_classified_metric_biased_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % Taiwan_classified_metric_biased_test.average_odds_difference())
print("Test set: Theil_index = %f" % Taiwan_classified_metric_biased_test.theil_index())

#### Plain model - without debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = 0.017283
Test set: Difference in mean outcomes between unprivileged and privileged groups = 0.022866


#### Plain model - without debiasing - classification metrics

Test set: Classification accuracy = 0.784979
Test set: Balanced classification accuracy = 0.689399
Test set: Disparate impact = 1.029817
Test set: Equal opportunity difference = 0.008847
Test set: Average odds difference = 0.024816
Test set: Theil_index = 0.144883


In [14]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

In [15]:
debiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=True,
                          sess=sess)

In [16]:
debiased_model.fit(Taiwan_bld_train)

epoch 0; iter: 0; batch classifier loss: 0.674932; batch adversarial loss: 0.203929
epoch 1; iter: 0; batch classifier loss: 2.030119; batch adversarial loss: 0.497182
epoch 2; iter: 0; batch classifier loss: 2.373574; batch adversarial loss: 0.303297
epoch 3; iter: 0; batch classifier loss: 2.393769; batch adversarial loss: 0.137478
epoch 4; iter: 0; batch classifier loss: 2.640889; batch adversarial loss: -0.039107
epoch 5; iter: 0; batch classifier loss: 2.660581; batch adversarial loss: -0.161785
epoch 6; iter: 0; batch classifier loss: 2.620665; batch adversarial loss: -0.200813
epoch 7; iter: 0; batch classifier loss: 3.066199; batch adversarial loss: -0.462447
epoch 8; iter: 0; batch classifier loss: 3.258683; batch adversarial loss: -0.443979
epoch 9; iter: 0; batch classifier loss: 3.977088; batch adversarial loss: -0.718088
epoch 10; iter: 0; batch classifier loss: 3.963078; batch adversarial loss: -0.804861
epoch 11; iter: 0; batch classifier loss: 3.847316; batch adversaria

In [17]:
Taiwan_debiased_train = debiased_model.predict(Taiwan_bld_train)
Taiwan_debiased_test  = debiased_model.predict(Taiwan_bld_test)

In [19]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - after debiasing - dataset metrics"))
Taiwan_metric_debiased_train = BinaryLabelDatasetMetric(Taiwan_debiased_train, 
                               unprivileged_groups=unprivileged_groups,
                               privileged_groups=privileged_groups)

print("Train set: Difference in mean outcomes between unprivileged and privileged groups = %f" % Taiwan_metric_debiased_train.mean_difference())

Taiwan_metric_debiased_test = BinaryLabelDatasetMetric(Taiwan_debiased_test, 
                              unprivileged_groups=unprivileged_groups,
                              privileged_groups=privileged_groups)

print("Test set: Difference in mean outcomes between unprivileged and privileged groups = %f" % Taiwan_metric_debiased_test.mean_difference())

display(Markdown("#### Plain model - after debiasing - classification metrics"))
Taiwan_classified_metric_debiased_test = ClassificationMetric(Taiwan_bld_test,Taiwan_debiased_test,
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)
print("Test set: Classification accuracy = %f" % Taiwan_classified_metric_debiased_test.accuracy())
TPR = Taiwan_classified_metric_debiased_test.true_positive_rate()
TNR = Taiwan_classified_metric_debiased_test.true_negative_rate()
bal_acc_debiased_test = 0.5*(TPR+TNR)
print("Test set: Balanced classification accuracy = %f" % bal_acc_debiased_test)
print("Test set: Disparate impact = %f" % Taiwan_classified_metric_debiased_test.disparate_impact())
print("Test set: Equal opportunity difference = %f" % Taiwan_classified_metric_debiased_test.equal_opportunity_difference())
print("Test set: Average odds difference = %f" % Taiwan_classified_metric_debiased_test.average_odds_difference())
print("Test set: Theil_index = %f" % Taiwan_classified_metric_debiased_test.theil_index())

#### Plain model - after debiasing - dataset metrics

Train set: Difference in mean outcomes between unprivileged and privileged groups = -0.845176
Test set: Difference in mean outcomes between unprivileged and privileged groups = -0.053965


#### Plain model - after debiasing - classification metrics

Test set: Classification accuracy = 0.250295
Test set: Balanced classification accuracy = 0.509911
Test set: Disparate impact = 0.000000
Test set: Equal opportunity difference = -0.064672
Test set: Average odds difference = -0.042305
Test set: Theil_index = 1.379676
